In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import json
import time
from tqdm.notebook import tqdm 

<h4>Start cells here if {state}_geo file has not been FIPS encoded

In [4]:
fips_check = pd.read_csv("data/state_data/geo/geocoded/geo_md.csv") #- original geocoded file
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1"], axis =1) 
fips_check.head() #18979

,Unnamed: 0,Unnamed: 0.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,...,Veteran,NonProfit,ForgivenessAmount,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long
0,425853,425853,4142717108,4/12/20,373,PPP,"PT NETWORK, LLC",501 Fairmount Ave,Towson,MD,...,Non-Veteran,NaN,10114520.55,6/11/21,62.0,Health Care and Social Assistance,BALTIMORE DISTRICT OFFICE,"501 Fairmount Ave, Towson, MD",39.400438,-76.597013
1,425854,425854,4682157009,4/4/20,373,PPP,"GMS MINE REPAIR AND MAINTENANCE, INC.",PO Box 2446 32 Enterprise Drive,OAKLAND,MD,...,Veteran,NaN,10118611.11,6/15/21,21.0,Mining,BALTIMORE DISTRICT OFFICE,"PO Box 2446 32 Enterprise Drive, OAKLAND, MD",39.406150,-79.355064
2,425855,425855,8298497108,4/15/20,373,PPP,"GSE SYSTEMS, INC.",1332 Londontown Blvd 200,Sykesville,MD,...,Unanswered,NaN,10126849.32,8/2/21,33.0,Manufacturing,BALTIMORE DISTRICT OFFICE,"1332 Londontown Blvd 200, Sykesville, MD",39.407222,-76.944486
3,425856,425856,8415647105,4/15/20,373,PPP,"MISSION BBQ MANAGEMENT, LLC",7750 Governor Ritchie Highway,Glen Burnie,MD,...,Unanswered,NaN,10113698.63,6/11/21,72.0,Accomodation and Food Services,BALTIMORE DISTRICT OFFICE,"7750 Governor Ritchie Highway, Glen Burnie, MD",39.151658,-76.613993
4,425857,425857,9405257009,4/9/20,373,PPP,LEMEK LLC,8184 Lark Brown Road Suite 201,ELKRIDGE,MD,...,Non-Veteran,NaN,NaN,NaN,72.0,Accomodation and Food Services,BALTIMORE DISTRICT OFFICE,"8184 Lark Brown Road Suite 201, ELKRIDGE, MD",39.189665,-76.794909


In [5]:
columns = ['LoanNumber','full_add', 'Lat', 'Long']
fips_check = fips_check[columns]

In [6]:
fips_check_nan = fips_check[fips_check["Lat"].isnull()] # check to make sure no nulls in coords
fips_check_nan #0

,LoanNumber,full_add,Lat,Long


In [8]:
fips_check_notna = fips_check[fips_check["Lat"].notna()] # check to make sure no nulls in coords
fips_check_notna.head() #18979

,LoanNumber,full_add,Lat,Long
0,4142717108,"501 Fairmount Ave, Towson, MD",39.400438,-76.597013
1,4682157009,"PO Box 2446 32 Enterprise Drive, OAKLAND, MD",39.406150,-79.355064
2,8298497108,"1332 Londontown Blvd 200, Sykesville, MD",39.407222,-76.944486
3,8415647105,"7750 Governor Ritchie Highway, Glen Burnie, MD",39.151658,-76.613993
4,9405257009,"8184 Lark Brown Road Suite 201, ELKRIDGE, MD",39.189665,-76.794909


<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [ ]:
fips_check_scraped = pd.read_csv("data/state_data/geo/geo_fips/MD_fips_scraped.csv") 
fips_check_scraped.head() #22826

In [9]:
columns = ['LoanNumber','full_add', 'Lat', 'Long', 'FIPS_z']
fips_check_scraped = fips_check_scraped[columns]

In [10]:
#pd.set_option('display.float_format', '{:.0f}'.format)

In [ ]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna #22815

In [ ]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #2674

<h4>FIPS Script

In [9]:
#if geocode file
lat = fips_check_notna["Lat"].astype(float).astype(str).to_list()
long = fips_check_notna["Long"].astype(float).astype(str).to_list()

#if FIPS_z file 
#lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
#long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [10]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
    
executable_path = {'executable_path': ChromeDriverManager().install()}
driver = Browser('chrome', **executable_path, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('_x_id').clear()
        driver.find_by_id('_y_id').clear()
        driver.find_by_id('_x_id').fill(long[i])
        driver.find_by_id('_y_id').fill(lat[i])
        driver.find_by_id('proceed').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='pl_gov_census_geo_geocoder_domain_GeographyResult')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('24')] #MD
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                notfound.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1



====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome
Driver [/Users/ciaraspencer/.wdm/drivers/chromedriver/mac64/98.0.4758.80/chromedriver] found in cache


coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [11]:
geo_id_list =geo_id 

Length Check

In [12]:
print(len(geo_id_list))

18979


Matching FIPS to DataFrame

In [13]:
#for geocode file
fips_fill = fips_check_notna #confirm pair to notna DataFrame
#if FIPS available
#fips_fill = fips_check_scraped_null.iloc[:2674]

In [14]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

,LoanNumber,full_add,Lat,Long,FIPS_z
0,4142717108,"501 Fairmount Ave, Towson, MD",39.400438,-76.597013,[240054909002002]
1,4682157009,"PO Box 2446 32 Enterprise Drive, OAKLAND, MD",39.406150,-79.355064,[240230007004094]
2,8298497108,"1332 Londontown Blvd 200, Sykesville, MD",39.407222,-76.944486,[240135051021008]
3,8415647105,"7750 Governor Ritchie Highway, Glen Burnie, MD",39.151658,-76.613993,[240037304041001]
4,9405257009,"8184 Lark Brown Road Suite 201, ELKRIDGE, MD",39.189665,-76.794909,[240276067061004]
...,...,...,...,...,...
18974,8964248309,"7401 Woodmont Ave, Bethesda, MD",38.983328,-77.094790,[240317048041002]
18975,9164838305,"411 Aviation Way Ste 220, Frederick, MD",39.418681,-77.380239,[240217722002005]
18976,9216407410,"6537 BALTIMORE NATIONAL PIKE, CATONSVILLE, MD",39.285835,-76.765583,[240054015031001]
18977,9305188501,"32F Queen Anne Way, Chester, MD",38.984626,-76.280169,[240358108003004]


Remove brackets

In [15]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

,LoanNumber,full_add,Lat,Long,FIPS_z
0,4142717108,"501 Fairmount Ave, Towson, MD",39.400438,-76.597013,240054909002002
1,4682157009,"PO Box 2446 32 Enterprise Drive, OAKLAND, MD",39.406150,-79.355064,240230007004094
2,8298497108,"1332 Londontown Blvd 200, Sykesville, MD",39.407222,-76.944486,240135051021008
3,8415647105,"7750 Governor Ritchie Highway, Glen Burnie, MD",39.151658,-76.613993,240037304041001
4,9405257009,"8184 Lark Brown Road Suite 201, ELKRIDGE, MD",39.189665,-76.794909,240276067061004
...,...,...,...,...,...
18974,8964248309,"7401 Woodmont Ave, Bethesda, MD",38.983328,-77.094790,240317048041002
18975,9164838305,"411 Aviation Way Ste 220, Frederick, MD",39.418681,-77.380239,240217722002005
18976,9216407410,"6537 BALTIMORE NATIONAL PIKE, CATONSVILLE, MD",39.285835,-76.765583,240054015031001
18977,9305188501,"32F Queen Anne Way, Chester, MD",38.984626,-76.280169,240358108003004


In [16]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #16

,LoanNumber,full_add,Lat,Long,FIPS_z
458,5288307405,"821 kennedy st nw, washington, MD",38.956476,-77.025084,NaN
2032,4365897102,"1000 Harborview Ct., SUDLERSVILLE, MD",41.603221,-73.087749,NaN
2072,9453537004,"201 Le Phillip Ct NE, CHEVY CHASE, MD",41.603221,-73.087749,NaN
5240,2085197209,"9111 Edmonston Road, Suite 303, Greenbelt, MD",38.902914,-77.033066,NaN
9355,4458978805,"7404 Executive Pl Ste 400, Lanham, MD",38.992494,-76.825910,NaN
10959,5522567107,"5594 ASHLAND COMMUNITY SQ, EDGEWATER, MD",38.613268,-77.373759,NaN
13028,3652408307,"12614 Nicklaus Ct, Cumberland, MD",41.603221,-73.087749,NaN
14558,5893948904,"4075 Wilson Blvd STE 800, Arlington, MD",38.880246,-77.108616,NaN
14752,6542178400,"2807 Bel Ct, Manchester, MD",41.603221,-73.087749,NaN
14968,6862267700,"2807 BEL CT, MANCHESTER, MD",41.603221,-73.087749,NaN


Parse for Merge

In [17]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
0,4142717108,240054909002002
1,4682157009,240230007004094
2,8298497108,240135051021008
3,8415647105,240037304041001
4,9405257009,240276067061004
...,...,...
18974,8964248309,240317048041002
18975,9164838305,240217722002005
18976,9216407410,240054015031001
18977,9305188501,240358108003004


Read in base data

In [19]:
#if from geocode file
fips_check = pd.read_csv("data/state_data/geo/geocoded/geo_md.csv")
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1"], axis =1) 
fips_check

#if from FIPS file
#fips_check = pd.read_csv("data/state_data/geo/geo_fips/MD_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 
#fips_check.head() #22826


,Unnamed: 0,Unnamed: 0.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,...,Veteran,NonProfit,ForgivenessAmount,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long
0,425853,425853,4142717108,4/12/20,373,PPP,"PT NETWORK, LLC",501 Fairmount Ave,Towson,MD,...,Non-Veteran,NaN,10114520.55,6/11/21,62.0,Health Care and Social Assistance,BALTIMORE DISTRICT OFFICE,"501 Fairmount Ave, Towson, MD",39.400438,-76.597013
1,425854,425854,4682157009,4/4/20,373,PPP,"GMS MINE REPAIR AND MAINTENANCE, INC.",PO Box 2446 32 Enterprise Drive,OAKLAND,MD,...,Veteran,NaN,10118611.11,6/15/21,21.0,Mining,BALTIMORE DISTRICT OFFICE,"PO Box 2446 32 Enterprise Drive, OAKLAND, MD",39.406150,-79.355064
2,425855,425855,8298497108,4/15/20,373,PPP,"GSE SYSTEMS, INC.",1332 Londontown Blvd 200,Sykesville,MD,...,Unanswered,NaN,10126849.32,8/2/21,33.0,Manufacturing,BALTIMORE DISTRICT OFFICE,"1332 Londontown Blvd 200, Sykesville, MD",39.407222,-76.944486
3,425856,425856,8415647105,4/15/20,373,PPP,"MISSION BBQ MANAGEMENT, LLC",7750 Governor Ritchie Highway,Glen Burnie,MD,...,Unanswered,NaN,10113698.63,6/11/21,72.0,Accomodation and Food Services,BALTIMORE DISTRICT OFFICE,"7750 Governor Ritchie Highway, Glen Burnie, MD",39.151658,-76.613993
4,425857,425857,9405257009,4/9/20,373,PPP,LEMEK LLC,8184 Lark Brown Road Suite 201,ELKRIDGE,MD,...,Non-Veteran,NaN,NaN,NaN,72.0,Accomodation and Food Services,BALTIMORE DISTRICT OFFICE,"8184 Lark Brown Road Suite 201, ELKRIDGE, MD",39.189665,-76.794909
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18974,444827,444827,8964248309,1/30/21,353,PPS,CESCO LLC,7401 Woodmont Ave,Bethesda,MD,...,Unanswered,NaN,151079.17,11/3/21,72.0,Accomodation and Food Services,WASHINGTON DISTRICT OFFICE,"7401 Woodmont Ave, Bethesda, MD",38.983328,-77.094790
18975,444828,444828,9164838305,1/30/21,373,PPS,D&B MEDICAL MANAGEMENT SOLUTIONS INC.,411 Aviation Way Ste 220,Frederick,MD,...,Non-Veteran,NaN,NaN,NaN,54.0,"Professional, Scientific, and Technical Services",BALTIMORE DISTRICT OFFICE,"411 Aviation Way Ste 220, Frederick, MD",39.418681,-77.380239
18976,444829,444829,9216407410,5/19/20,373,PPP,MELROY PLUMBING & HEATING INC,6537 BALTIMORE NATIONAL PIKE,CATONSVILLE,MD,...,Unanswered,NaN,122951.48,9/13/21,23.0,Construction,BALTIMORE DISTRICT OFFICE,"6537 BALTIMORE NATIONAL PIKE, CATONSVILLE, MD",39.285835,-76.765583
18977,444830,444830,9305188501,3/12/21,373,PPS,EVANS DELIVERY SERVICE INC,32F Queen Anne Way,Chester,MD,...,Non-Veteran,NaN,NaN,NaN,49.0,Transportation and Warehousing,BALTIMORE DISTRICT OFFICE,"32F Queen Anne Way, Chester, MD",38.984626,-76.280169


Merge with MainFrame

In [20]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,Unnamed: 0.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,...,NonProfit,ForgivenessAmount,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long,FIPS_z
0,425853,425853,4142717108,4/12/20,373,PPP,"PT NETWORK, LLC",501 Fairmount Ave,Towson,MD,...,NaN,10114520.55,6/11/21,62.0,Health Care and Social Assistance,BALTIMORE DISTRICT OFFICE,"501 Fairmount Ave, Towson, MD",39.400438,-76.597013,240054909002002
1,425854,425854,4682157009,4/4/20,373,PPP,"GMS MINE REPAIR AND MAINTENANCE, INC.",PO Box 2446 32 Enterprise Drive,OAKLAND,MD,...,NaN,10118611.11,6/15/21,21.0,Mining,BALTIMORE DISTRICT OFFICE,"PO Box 2446 32 Enterprise Drive, OAKLAND, MD",39.406150,-79.355064,240230007004094
2,425855,425855,8298497108,4/15/20,373,PPP,"GSE SYSTEMS, INC.",1332 Londontown Blvd 200,Sykesville,MD,...,NaN,10126849.32,8/2/21,33.0,Manufacturing,BALTIMORE DISTRICT OFFICE,"1332 Londontown Blvd 200, Sykesville, MD",39.407222,-76.944486,240135051021008
3,425856,425856,8415647105,4/15/20,373,PPP,"MISSION BBQ MANAGEMENT, LLC",7750 Governor Ritchie Highway,Glen Burnie,MD,...,NaN,10113698.63,6/11/21,72.0,Accomodation and Food Services,BALTIMORE DISTRICT OFFICE,"7750 Governor Ritchie Highway, Glen Burnie, MD",39.151658,-76.613993,240037304041001
4,425857,425857,9405257009,4/9/20,373,PPP,LEMEK LLC,8184 Lark Brown Road Suite 201,ELKRIDGE,MD,...,NaN,NaN,NaN,72.0,Accomodation and Food Services,BALTIMORE DISTRICT OFFICE,"8184 Lark Brown Road Suite 201, ELKRIDGE, MD",39.189665,-76.794909,240276067061004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18974,444827,444827,8964248309,1/30/21,353,PPS,CESCO LLC,7401 Woodmont Ave,Bethesda,MD,...,NaN,151079.17,11/3/21,72.0,Accomodation and Food Services,WASHINGTON DISTRICT OFFICE,"7401 Woodmont Ave, Bethesda, MD",38.983328,-77.094790,240317048041002
18975,444828,444828,9164838305,1/30/21,373,PPS,D&B MEDICAL MANAGEMENT SOLUTIONS INC.,411 Aviation Way Ste 220,Frederick,MD,...,NaN,NaN,NaN,54.0,"Professional, Scientific, and Technical Services",BALTIMORE DISTRICT OFFICE,"411 Aviation Way Ste 220, Frederick, MD",39.418681,-77.380239,240217722002005
18976,444829,444829,9216407410,5/19/20,373,PPP,MELROY PLUMBING & HEATING INC,6537 BALTIMORE NATIONAL PIKE,CATONSVILLE,MD,...,NaN,122951.48,9/13/21,23.0,Construction,BALTIMORE DISTRICT OFFICE,"6537 BALTIMORE NATIONAL PIKE, CATONSVILLE, MD",39.285835,-76.765583,240054015031001
18977,444830,444830,9305188501,3/12/21,373,PPS,EVANS DELIVERY SERVICE INC,32F Queen Anne Way,Chester,MD,...,NaN,NaN,NaN,49.0,Transportation and Warehousing,BALTIMORE DISTRICT OFFICE,"32F Queen Anne Way, Chester, MD",38.984626,-76.280169,240358108003004


<b>Fill FIPS Columns - if FIPS file

In [ ]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge

Drop Extra FIPS Columns

In [ ]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0.1", "Unnamed: 0.1.1.1"], axis=1)
fips_merge_drop

In [ ]:
#fips_merge_drop = fips_merge.drop(["Unnamed: 0.1", "Unnamed: 0.1.1.1"], axis=1)
fips_merge_drop.loc[fips_merge["FIPS_z"].isnull()] #verify #27 , notna #22799 / 22826

In [21]:
fips_MD = fips_merge
#fips_MD = fips_merge_drop

Convert to File

In [5]:
fips_MD.to_csv("data/state_data/geo/geo_fips/MD_fips_scraped.csv") 

Review & Compare

In [2]:
geo_MD_fips = pd.read_csv("data/state_data/geo/geo_fips/MD_fips_scraped.csv") 
geo_MD_fips.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,...,NonProfit,ForgivenessAmount,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long,FIPS_z
0,0,0,0,0,425853,425853,4142717108,4/12/20,373,PPP,...,N,10114520.55,6/11/21,62.0,Health Care and Social Assistance,BALTIMORE DISTRICT OFFICE,"501 Fairmount Ave, Towson, MD",39.400438,-76.597013,2.400549e+14
1,1,1,1,1,425854,425854,4682157009,4/4/20,373,PPP,...,N,10118611.11,6/15/21,21.0,Mining,BALTIMORE DISTRICT OFFICE,"PO Box 2446 32 Enterprise Drive, OAKLAND, MD",39.406150,-79.355064,2.402300e+14
2,2,2,2,2,425855,425855,8298497108,4/15/20,373,PPP,...,N,10126849.32,8/2/21,33.0,Manufacturing,BALTIMORE DISTRICT OFFICE,"1332 Londontown Blvd 200, Sykesville, MD",39.407222,-76.944486,2.401351e+14
3,3,3,3,3,425856,425856,8415647105,4/15/20,373,PPP,...,N,10113698.63,6/11/21,72.0,Accomodation and Food Services,BALTIMORE DISTRICT OFFICE,"7750 Governor Ritchie Highway, Glen Burnie, MD",39.151658,-76.613993,2.400373e+14
4,4,4,4,4,425857,425857,9405257009,4/9/20,373,PPP,...,N,NaN,NaN,72.0,Accomodation and Food Services,BALTIMORE DISTRICT OFFICE,"8184 Lark Brown Road Suite 201, ELKRIDGE, MD",39.189665,-76.794909,2.402761e+14


In [3]:
##### fill in missing Lat, Long, FIPS
geo_MD_fips.loc[geo_MD_fips["FIPS_z"].isnull()] #27

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,...,NonProfit,ForgivenessAmount,ForgivenessDate,NA,Industry,SBAOfficeLabel,full_add,Lat,Long,FIPS_z
458,458,458,458,426311,426311,5288307405,5/12/20,353,PPP,ABA,...,N,2360747.54,9/8/21,62.0,Health Care and Social Assistance,WASHINGTON DISTRICT OFFICE,"821 kennedy st nw, washington, MD",38.956476,-77.025084,NaN
2032,2032,2032,2032,427885,427885,4365897102,4/13/20,373,PPP,THE HARBOR SALES COMPANY,...,N,1058167.89,11/17/20,42.0,Wholesale Trade,BALTIMORE DISTRICT OFFICE,"1000 Harborview Ct., SUDLERSVILLE, MD",41.603221,-73.087749,NaN
2072,2072,2072,2072,427925,427925,9453537004,4/9/20,353,PPP,"CABARRUS EYE CENTER, PA",...,N,1052797.54,5/20/21,62.0,Health Care and Social Assistance,WASHINGTON DISTRICT OFFICE,"201 Le Phillip Ct NE, CHEVY CHASE, MD",41.603221,-73.087749,NaN
5240,5240,5240,5240,431093,431093,2085197209,4/15/20,353,PPP,"N2GRATE GOVERRNMENT TECHNOLOGY SOLUTIONS, LLC",...,N,506051.99,11/3/20,45.0,Retail Trade,WASHINGTON DISTRICT OFFICE,"9111 Edmonston Road, Suite 303, Greenbelt, MD",38.902914,-77.033066,NaN
9355,9355,9355,9355,435208,435208,4458978805,4/16/21,353,PPP,OLIVIA EVANS EL,...,N,NaN,NaN,54.0,"Professional, Scientific, and Technical Services",WASHINGTON DISTRICT OFFICE,"7404 Executive Pl Ste 400, Lanham, MD",38.992494,-76.825910,NaN
10959,10959,10959,10959,436812,436812,5522567107,4/13/20,373,PPP,NV STEAKHOUSE LLC,...,N,265746.34,6/10/21,53.0,Real Estate Rental and Leasing,BALTIMORE DISTRICT OFFICE,"5594 ASHLAND COMMUNITY SQ, EDGEWATER, MD",38.613268,-77.373759,NaN
13028,13028,13028,13028,438881,438881,3652408307,1/22/21,373,PPS,U.S. MOBILE IMAGING LLC,...,N,224024.23,8/30/21,62.0,Health Care and Social Assistance,BALTIMORE DISTRICT OFFICE,"12614 Nicklaus Ct, Cumberland, MD",41.603221,-73.087749,NaN
14558,14558,14558,14558,440411,440411,5893948904,5/1/21,373,PPP,CYNNOVATIVE LLC,...,N,200466.55,10/27/21,54.0,"Professional, Scientific, and Technical Services",BALTIMORE DISTRICT OFFICE,"4075 Wilson Blvd STE 800, Arlington, MD",38.880246,-77.108616,NaN
14752,14752,14752,14752,440605,440605,6542178400,2/10/21,373,PPS,LITTLE PEOPLES PLACE CHILD DEVELOPMENT CENTER ...,...,N,197764.34,8/18/21,54.0,"Professional, Scientific, and Technical Services",BALTIMORE DISTRICT OFFICE,"2807 Bel Ct, Manchester, MD",41.603221,-73.087749,NaN
14968,14968,14968,14968,440821,440821,6862267700,5/1/20,373,PPP,LITTLE PEOPLES PLACE CHILD DEVELOPMENT CENTER ...,...,N,195546.61,2/18/21,62.0,Health Care and Social Assistance,BALTIMORE DISTRICT OFFICE,"2807 BEL CT, MANCHESTER, MD",41.603221,-73.087749,NaN


In [4]:
geo_MD_fips["BusinessType"] = geo_MD_fips["BusinessType"].replace({'Limited  Liability Company(LLC)': 'Limited Liability Company(LLC)'})
geo_MD_fips.to_csv("data/state_data/geo/geo_fips/MD_fips_scraped.csv")
listbusiness = geo_MD_fips["BusinessType"].value_counts()
listbusiness

Corporation                            7687
Limited Liability Company(LLC)         5488
Subchapter S Corporation               3719
Non-Profit Organization                1253
Sole Proprietorship                     222
Partnership                             205
Limited Liability Partnership           145
Professional Association                142
Cooperative                              28
501(c)3 – Non Profit                     27
Self-Employed Individuals                26
501(c)6 – Non Profit Membership          13
Non-Profit Childcare Center               6
Trust                                     6
Employee Stock Ownership Plan(ESOP)       5
Single Member LLC                         3
Not Available                             2
Tenant in Common                          1
Housing Co-op                             1
Name: BusinessType, dtype: int64